In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!python --version
!uname -a

In [ ]:
## FROM GLM ##
# CELL 1: INSTALL DEPENDENCIES
!pip install fastapi uvicorn pyngrok transformers accelerate bitsandbytes

In [2]:
## HF Login ##
from huggingface_hub import login, whoami
from kaggle_secrets import UserSecretsClient

# 1. Retrieve the secret from Kaggle
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")

# 2. Log in programmatically
if hf_token:
    #login(token=hf_token, add_to_git_credential=True)
    login(token=hf_token)
    # 3. Verify and display login status
    try:
        user_info = whoami()
        print(f"✅ Success! Logged in as: {user_info['name']}")
        if user_info.get('orgs'):
            print(f"🏢 Organizations: {', '.join([org['name'] for org in user_info['orgs']])}")
    except Exception as e:
        print(f"❌ Login failed or token invalid: {e}")
else:
    print("⚠️ HF_TOKEN not found. Please add it to 'Add-ons > Secrets'.")



✅ Success! Logged in as: Shogun9000


In [4]:
# CELL 2: THE MASTER AI SCRIPT (ARTIFACT INTEGRATION)
import threading
import torch
import uvicorn
from fastapi import FastAPI, Request
from pyngrok import ngrok
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# --- CONFIGURATION ---
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# --- ARTIFACT DICTIONARY (GLOBAL LEVEL) ---
# Map simple names to your Roblox Asset IDs
ARTIFACT_IDS = {
    "cyber_obelisks": "rbxassetid://72574562363210",
    "data_keys": "rbxassetid://98821197258773",
    "circuit_walls": "rbxassetid://104990479943956"
}

# 1. INITIALIZE TOKENIZER & MODEL
print("[INIT] Loading Model...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    load_in_4bit=True,
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

def get_ai_master_response(game_data):
    system_msg = (
        "You are an Ancient Master of a Cyberpunk Angkor Wat Temple. "
        "You are surrounded by powerful Artifacts (Ancient Tech). \n"
        "Use ONLY these names: \n"
        "'cyber_obelisk(1)' (A glowing red obelisk). \n"
        "'data_key(1)' (A holographic golden key). \n"
        "'circuit_wall(1)' (An ancient wall with circuits). \n"
        "When the player's health is < 50%, "
        "you must summon an Artifact (Example: 'spawn_enemy': 'cyber_obelisk(1)'). "
        "If player is healthy, spawn a Cyber-Monkey. "
        "Respond ONLY with dialogue."
    )
    
    user_msg = f"Initiate is at {game_data['zone']} with {game_data['health']}% integrity."
    
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_msg},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    outputs = pipe(
        prompt, 
        max_new_tokens=64, 
        do_sample=True, 
        temperature=0.8,
        return_full_text=False
    )
    
    return outputs[0]

# --- API DEFINITION ---
app = FastAPI()

@app.post("/temple_update")
async def temple_handler(request: Request):
    data = await request.json()
    
    # 1. CLEAN UP DATA
    try:
        if isinstance(raw_ai, dict):
            final_voice = raw_ai.get('generated_text', raw_ai)
        else:
            final_voice = raw_ai
    except Exception as e:
        print(f"[ERROR PARSING]: {e}")
        final_voice = "My mind is clouded by static..."
    
    print(f"\n[CLEAN VOICE]: {final_voice}")

    # 2. TRANSLATE: Swap names for IDs
    # We lower-case the voice to make searching easier
    voice_lower = final_voice.lower()
    spawn_value = "Cyber_Monkey" # Default fallback
    
    if "cyber_obelisk" in voice_lower:
        spawn_value = ARTIFACT_IDS["cyber_obelisks"]
        print("[TRANSLATOR] Swapped 'cyber_obelisks' -> Obelisk ID")
    elif "data_key(1)" in voice_lower:
        spawn_value = ARTIFACT_IDS["data_keys"]
        print("[TRANSLATOR] Swapped 'data_keys' -> Key ID")
    elif "circuit_wall" in voice_lower:
        spawn_value = ARTIFACT_IDS["circuit_walls"]
        print("[TRANSLATOR] Swapped 'circuit_walls' -> Wall ID")

    # 3. DYNAMIC DECISION LOGIC
    player_health = data.get('health', 100)
    
    if player_health < 50:
        # CRITICAL HEALTH
        env_command = "RED_ALERT"
        print(f"[MASTER DECISION]: Player CRITICAL! Sending RED_ALERT & {spawn_value}")
    else:
        # HEALTHY
        env_command = "NEON_PULSE"
        print(f"[MASTER DECISION]: Player Safe. Sending NEON_PULSE")
    
    return {
        "master_voice": final_voice,
        "environment_trigger": env_command,
        "spawn_enemy": spawn_value,
        "status": 200
    }

# --- SERVER STARTUP ---
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="error")

# Start server in a background thread
threading.Thread(target=run_server, daemon=True).start()

# Connect ngrok
NGROK_TOKEN = "38rox33U2EK3Z7w0UYlYrV6me9Z_2t79GYbTgoJM5YDAoHPKV"
ngrok.set_auth_token(NGROK_TOKEN)
tunnel = ngrok.connect(8000)

print(f"\n[MASTER AWAKENED] 2x T4 GPUs ACTIVE")
print(f"URL: {tunnel.public_url}/temple_update")
print("------------------------------------------------")

[INIT] Loading Model...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): [errno 98] address already in use



[MASTER AWAKENED] 2x T4 GPUs ACTIVE
URL: https://olivia-decadal-preciously.ngrok-free.dev/temple_update
------------------------------------------------
[ERROR PARSING]: name 'raw_ai' is not defined

[CLEAN VOICE]: My mind is clouded by static...
[MASTER DECISION]: Player Safe. Sending NEON_PULSE
[ERROR PARSING]: name 'raw_ai' is not defined

[CLEAN VOICE]: My mind is clouded by static...
[MASTER DECISION]: Player Safe. Sending NEON_PULSE
[ERROR PARSING]: name 'raw_ai' is not defined

[CLEAN VOICE]: My mind is clouded by static...
[MASTER DECISION]: Player Safe. Sending NEON_PULSE
[ERROR PARSING]: name 'raw_ai' is not defined

[CLEAN VOICE]: My mind is clouded by static...
[MASTER DECISION]: Player Safe. Sending NEON_PULSE


In [1]:
# CELL 1: INSTALL & LOAD
#!pip install torch huggingface_hub transformers accelerate bitsandbytes
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# --- CONFIGURATION ---
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

print("[INIT] Loading Model...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    load_in_4bit=True,
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)
print("[INIT] Model Loaded Successfully.")

# --- ARTIFACT DICTIONARY ---
#ARTIFACT_IDS = {
#    "cyber_obelisks": "rbxassetid://72574562363210",
#    "data_keys": "rbxassetid://98821197258773",
#    "circuit_walls": "rbxassetid://104990479943956"
#}

2026-01-29 20:52:00.208695: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769719920.238137     214 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769719920.245629     214 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769719920.286875     214 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769719920.286901     214 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769719920.286903     214 computation_placer.cc:177] computation placer alr

[INIT] Loading Model...


`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


[INIT] Model Loaded Successfully.


In [ ]:
# CELL 2: THE SERVER (Must be run AFTER Cell 1)
# CELL 2: THE SERVER (Persona-Logic Fixed)
import threading
import uvicorn
from fastapi import FastAPI, Request
from pyngrok import ngrok

app = FastAPI()

ARTIFACT_IDS = {
    # Cyberpunk Zone
    "cyber_obelisk": "PASTE_NEW_ID_HERE",
    "data_key": "PASTE_NEW_ID_HERE",
    "circuit_wall": "PASTE_NEW_ID_HERE",
    "healing_crystal": "PASTE_NEW_ID_HERE",
    
    # Western Zone
    "restorative_tonic": "PASTE_NEW_ID_HERE",
    
    # Haunted Mansion Zone
    "penicillin": "PASTE_NEW_ID_HERE"
}

last_state = {"spawn": "", "health_mode": ""}

PERSONAS = {
    "Angkor_Wat": {
        "name": "Ancient Cyber-Master",
        "voice_style": "cryptic, technological, and spiritual. Refer to artifacts as 'ancient tech relics'.",
        "keywords": ["Integrity", "Artifact", "Static"]
    },
    "Western_Town": {
        "name": "Neon Desperado",
        "voice_style": "gritty, cowboy-slang, high-noon cyberpunk. Refer to artifacts as 'bounty' or 'stolen loot'.",
        "keywords": ["Outlaw", "Grit", "Tumbleweed"]
    },
    "Haunted_Mansion": {
        "name": "Digital Ghost",
        "voice_style": "wispy, terrifying, glitchy. Refer to artifacts as 'cursed code' or 'echoes'.",
        "keywords": ["Curse", "Shadow", "Glitch"]
    }
}

@app.post("/temple_update")
async def temple_handler(request: Request):
    global last_state
    data = await request.json()
    player_health = data.get('health', 100)
    current_zone = data.get("zone", "Angkor_Wat")
    
    theme = PERSONAS.get(current_zone, PERSONAS["Angkor_Wat"])

    try:
        # CONSTRUCT DYNAMIC PROMPT
        system_msg = (
            f"You are the {theme['name']}. Your style is {theme['voice_style']}. "
            f"Use ONLY these names: 'cyber_obelisks', 'data_keys', 'circuit_walls'. "
            f"Always use these terms: {', '.join(theme['keywords'])}. "
            "If health < 50%, mention one artifact to help. If healthy, mention a Cyber-Monkey. "
            "Respond ONLY with dialogue."
        )
        
        user_msg = f"Initiate is at {current_zone} with {player_health}% integrity."
        
        messages = [{"role": "system", "content": system_msg}, {"role": "user", "content": user_msg}]
        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
        outputs = pipe(prompt, max_new_tokens=64, do_sample=True, temperature=0.8, return_full_text=False)
        final_voice = outputs[0]['generated_text']
    
    except Exception as e:
        print(f"[ERROR]: {e}")
        final_voice = "The connection is flickering..."

    # TRANSLATE & COOLDOWN
    voice_lower = final_voice.lower()
    current_asset = ""
    if "cyber_obelisks" in voice_lower: current_asset = ARTIFACT_IDS["cyber_obelisks"]
    elif "data_keys" in voice_lower: current_asset = ARTIFACT_IDS["data_keys"]
    elif "circuit_walls" in voice_lower: current_asset = ARTIFACT_IDS["circuit_walls"]
    elif "monkey" in voice_lower: current_asset = "Cyber_Monkey"

    spawn_value = ""
    if current_asset != last_state["spawn"]:
        spawn_value = current_asset
        last_state["spawn"] = current_asset
    
    env_command = "RED_ALERT" if player_health < 50 else "NEON_PULSE"
    
    print(f"\n[ZONE: {current_zone}] [VOICE]: {final_voice}")
    
    return {"master_voice": final_voice, "environment_trigger": env_command, "spawn_enemy": spawn_value, "zone": current_zone}

# Ngrok/Uvicorn logic remains same as your previous script


# --- SERVER STARTUP ---
def run_server():
    # Use [Uvicorn](https://www.uvicorn.org) for the FastAPI ASGI server
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="error")

# Run in background
threading.Thread(target=run_server, daemon=True).start()

# Connect via [pyngrok](https://pypi.org)
NGROK_TOKEN = "38rox33U2EK3Z7w0UYlYrV6me9Z_2t79GYbTgoJM5YDAoHPKV"
ngrok.set_auth_token(NGROK_TOKEN)
tunnel = ngrok.connect(8000)

print(f"\n[MASTER AWAKENED] 2x T4 GPUs ACTIVE")
print(f"URL: {tunnel.public_url}/temple_update")


In [ ]:
# CELL 2: THE REVOLUTIONARY DUNGEON MASTER (Final Multi-Theme Fix)
import threading
import uvicorn
from fastapi import FastAPI, Request
from pyngrok import ngrok

app = FastAPI()

# 1. THE FOUNDATION LIBRARY (Update with your 6 New IDs)
ARTIFACT_IDS = {
    "cyber_obelisk": "127842603615343",
    "data_key": "74842590890986",
    "circuit_wall": "127842603615343",
    "healing_crystal": "116896619676569",
    "healing_canteen": "139325813882604", 
    "penecillin": "102871796120423"    
}

last_state = {"spawn": "", "health_mode": ""}

PERSONAS = {
    "Angkor_Wat": {
        "name": "Ancient Cyber-Master",
        "voice_style": "cryptic, technological, and spiritual. Refer to artifacts as 'ancient tech relics'.",
        "keywords": ["Integrity", "Artifact", "Static", "healing_crystal"]
    },
    "Western_Town": {
        "name": "Neon Desperado",
        "voice_style": "gritty, cowboy-slang, high-noon cyberpunk. Refer to artifacts as 'bounty'.",
        "keywords": ["Outlaw", "Grit", "Tumbleweed", "healing_canteen"]
    },
    "Haunted_Mansion": {
        "name": "Digital Ghost",
        "voice_style": "wispy, terrifying, glitchy. Refer to artifacts as 'cursed code' or 'echoes'.",
        "keywords": ["Curse", "Shadow", "Glitch", "penecillin"]
    }
}

@app.post("/temple_update")
async def temple_handler(request: Request):
    global last_state
    data = await request.json()
    player_health = data.get('health', 100)
    current_zone = data.get("zone", "Angkor_Wat")
    
    theme = PERSONAS.get(current_zone, PERSONAS["Angkor_Wat"])
    target_item = theme["keywords"][-1]

    try:
        # 1. THE SYSTEM PROMPT (Fixed & Simplified)
        system_msg = (
            f"You are the {theme['name']}. Style: {theme['voice_style']}. "
            f"If health < 50%, you MUST output exactly '[ITEM:{target_item}]'. "
            f"Otherwise, use '[ITEM:cyber-monkey]'. "
            "Respond with ONE short sentence of dialogue, then the [ITEM:...] tag."
        )
        
        user_msg = f"Initiate is at {current_zone} with {player_health}% integrity."
        
        messages = [{"role": "system", "content": system_msg}, {"role": "user", "content": user_msg}]
        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
        outputs = pipe(prompt, max_new_tokens=64, do_sample=True, temperature=0.8, return_full_text=False)
        final_voice = outputs[0]['generated_text']
    
        # 2. EXTRACT ITEM & CLEAN VOICE
        import re
        current_asset = ""
        item_match = re.search(r"\[ITEM:(.*?)\]", final_voice, re.IGNORECASE)
        
        if item_match:
            keyword = item_match.group(1).lower().strip()
            current_asset = ARTIFACT_IDS.get(keyword, "")
            if keyword == "cyber-monkey":
                current_asset = "Cyber_Monkey"
        
        # REMOVE THE [ITEM:...] tag from the voice so the player doesn't see it
        clean_voice = re.sub(r"\[ITEM:.*?\]", "", final_voice).strip()

    except Exception as e:
        print(f"[ERROR]: {e}")
        clean_voice = "The connection is flickering..."
        current_asset = ""

    # 3. COOLDOWN & RETURN
    spawn_value = ""
    if current_asset != last_state["spawn"]:
        spawn_value = current_asset
        last_state["spawn"] = current_asset
    
    env_command = "RED_ALERT" if player_health < 50 else "NEON_PULSE"
    
    print(f"\n[ZONE: {current_zone}] [VOICE]: {clean_voice}")
    if spawn_value: print(f"--- TRIGGERING SPAWN: {spawn_value} ---")
    
    return {
        "master_voice": clean_voice, 
        "environment_trigger": env_command, 
        "spawn_enemy": spawn_value, 
        "zone": current_zone
    }


# --- SERVER STARTUP ---
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="error")

threading.Thread(target=run_server, daemon=True).start()

NGROK_TOKEN = "38rox33U2EK3Z7w0UYlYrV6me9Z_2t79GYbTgoJM5YDAoHPKV"
ngrok.set_auth_token(NGROK_TOKEN)
tunnel = ngrok.connect(8000)

print(f"\n[MASTER AWAKENED] 2x T4 GPUs ACTIVE")
print(f"URL: {tunnel.public_url}/temple_update")



[ZONE: Angkor_Wat] [VOICE]: "System integrity confirmed, temporal resonance aligned. Ancient Tech Relics within proximity:


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



[ZONE: Angkor_Wat] [VOICE]: "The ancient tech relics whisper secrets to me, their energy resonating in harmony with my digital essence."
--- TRIGGERING SPAWN: Cyber_Monkey ---


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



[ZONE: Angkor_Wat] [VOICE]: "System sync not achieved, rebooting..."
--- TRIGGERING SPAWN: 116896619676569 ---


...


[ZONE: Haunted_Mansion] [VOICE]: The walls seem to whisper my presence, a chilling caress of static.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



[ZONE: Haunted_Mansion] [VOICE]: The mansion's darkness seems to writhe like a living thing around me...
--- TRIGGERING SPAWN: 102871796120423 ---


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



[ZONE: Haunted_Mansion] [VOICE]: "Echoes of forgotten screams pierce the air..."
